In [ ]:
# ==== CELL 1: INSTALL & SETUP ====
# Thêm 'torchvision' vào đây để đồng bộ phiên bản với torch và torchaudio
!pip install -q "TTS>=0.22.0" "torch<2.6" torchaudio torchvision transformers datasets accelerate torchcodec

!pip install -q librosa soundfile pyarrow pyyaml tqdm

!apt-get update -qq && apt-get install -y espeak-ng ffmpeg

import os, torch, warnings, pathlib
warnings.filterwarnings('ignore')

# Kaggle cache dirs (giảm lỗi out-of-quota / write-permission)
os.environ["HF_HOME"] = "/kaggle/working/hf_home"
os.environ["HF_DATASETS_CACHE"] = "/kaggle/working/hf_datasets"
os.environ["HUGGINGFACE_HUB_CACHE"] = "/kaggle/working/hf_hub"
for d in ["/kaggle/working/hf_home", "/kaggle/working/hf_datasets", "/kaggle/working/hf_hub"]:
    pathlib.Path(d).mkdir(parents=True, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")
print("✅ Setup done.")

In [ ]:
# ==== CELL 3: SHARD DATA PREP ====
import os, gc, numpy as np, pyarrow.parquet as pq, soundfile as sf, librosa
from io import BytesIO
from huggingface_hub import hf_hub_download
from tqdm import tqdm

REPO_ID = "NhutP/VietSpeech"                     # Dataset HF (đúng theo bạn nêu)
DATASET_DIR = "/kaggle/working/xtts_dataset"     # Thư mục cố định cho XTTS
WAV_DIR = f"{DATASET_DIR}/wavs"
os.makedirs(WAV_DIR, exist_ok=True)

def build_metadata_for_shard(
    shard_idx: int,
    repo_id: str = REPO_ID,
    max_samples: int = 2000,     # số mẫu mỗi shard (tùy VRAM/dung lượng)
    min_dur: float = 1.0,
    max_dur: float = 15.0,
    target_sr: int = 22050
) -> int:
    """
    - Tải 1 shard parquet -> giải mã audio hợp lệ -> lưu WAV 16-bit mono 22.05kHz
    - Ghi ĐÈ metadata.csv (format XTTS): audio_file|text|speaker_name|language
    - Trả về số mẫu đã ghi
    """
    # Xóa WAV cũ để trống chỗ
    for fn in os.listdir(WAV_DIR):
        try: os.remove(os.path.join(WAV_DIR, fn))
        except: pass

    metadata_lines = []
    written = 0

    filename = f"data/train-{shard_idx:05d}-of-00027.parquet"
    local_path = None
    try:
        print(f"\n📥 Shard {shard_idx+1}/27 → {filename}")
        local_path = hf_hub_download(repo_id=repo_id, filename=filename, repo_type="dataset")
        pf = pq.ParquetFile(local_path)

        for batch in pf.iter_batches(batch_size=256):
            batch_dict = batch.to_pydict()
            n = len(batch_dict["audio"])
            for i in range(n):
                if written >= max_samples: break
                try:
                    audio_item = batch_dict["audio"][i]
                    audio_bytes = audio_item["bytes"]
                    wav, sr = sf.read(BytesIO(audio_bytes))  # np.float

                    dur = len(wav)/sr
                    if dur < min_dur or dur > max_dur:
                        continue

                    # resample & mono
                    if sr != target_sr:
                        wav = librosa.resample(wav, orig_sr=sr, target_sr=target_sr)
                    if wav.ndim > 1:
                        wav = wav.mean(axis=1)

                    # normalize & quantize PCM_16 (tiết kiệm dung lượng)
                    if np.max(np.abs(wav)) > 0:
                        wav = wav/np.max(np.abs(wav))*0.95
                    wav_i16 = (wav * 32767.0).astype(np.int16)

                    # text
                    text = batch_dict.get("transcription", [None])[i] or batch_dict.get("text", [None])[i] or ""
                    text = text.strip()
                    if not (3 <= len(text) <= 500):
                        continue

                    # save wav
                    out_name = f"vi_{shard_idx:02d}_{written:06d}.wav"
                    out_path = os.path.join(WAV_DIR, out_name)
                    sf.write(out_path, wav_i16, target_sr, subtype="PCM_16")

                    # metadata row (XTTS: audio|text|speaker|language)
                    speaker = f"spk_{shard_idx:02d}"
                    metadata_lines.append(f"{out_name}|{text}|{speaker}|vi")

                    written += 1
                except Exception:
                    continue
            if written >= max_samples:
                break
    finally:
        if local_path and os.path.exists(local_path):
            try: os.remove(local_path)
            except: pass
        gc.collect()

    meta_path = os.path.join(DATASET_DIR, "metadata.csv")
    with open(meta_path, "w", encoding="utf-8") as f:
        f.write("\n".join(metadata_lines))

    print(f"✅ Shard {shard_idx+1}: wrote {written} samples → {meta_path}")
    return written

print("✅ Shard prep functions ready.")


In [ ]:
# ==== CELL 3: SHARD DATA PREP ====
import os, gc, numpy as np, pyarrow.parquet as pq, soundfile as sf, librosa
from io import BytesIO
from huggingface_hub import hf_hub_download
from tqdm import tqdm

REPO_ID = "NhutP/VietSpeech"                     # Dataset HF (đúng theo bạn nêu)
DATASET_DIR = "/kaggle/working/xtts_dataset"     # Thư mục cố định cho XTTS
WAV_DIR = f"{DATASET_DIR}/wavs"
os.makedirs(WAV_DIR, exist_ok=True)

def build_metadata_for_shard(
    shard_idx: int,
    repo_id: str = REPO_ID,
    max_samples: int = 2000,     # số mẫu mỗi shard (tùy VRAM/dung lượng)
    min_dur: float = 1.0,
    max_dur: float = 15.0,
    target_sr: int = 22050
) -> int:
    """
    - Tải 1 shard parquet -> giải mã audio hợp lệ -> lưu WAV 16-bit mono 22.05kHz
    - Ghi ĐÈ metadata.csv (format XTTS): audio_file|text|speaker_name|language
    - Trả về số mẫu đã ghi
    """
    # Xóa WAV cũ để trống chỗ
    for fn in os.listdir(WAV_DIR):
        try: os.remove(os.path.join(WAV_DIR, fn))
        except: pass

    metadata_lines = []
    written = 0

    filename = f"data/train-{shard_idx:05d}-of-00027.parquet"
    local_path = None
    try:
        print(f"\n📥 Shard {shard_idx+1}/27 → {filename}")
        local_path = hf_hub_download(repo_id=repo_id, filename=filename, repo_type="dataset")
        pf = pq.ParquetFile(local_path)

        for batch in pf.iter_batches(batch_size=256):
            batch_dict = batch.to_pydict()
            n = len(batch_dict["audio"])
            for i in range(n):
                if written >= max_samples: break
                try:
                    audio_item = batch_dict["audio"][i]
                    audio_bytes = audio_item["bytes"]
                    wav, sr = sf.read(BytesIO(audio_bytes))  # np.float

                    dur = len(wav)/sr
                    if dur < min_dur or dur > max_dur:
                        continue

                    # resample & mono
                    if sr != target_sr:
                        wav = librosa.resample(wav, orig_sr=sr, target_sr=target_sr)
                    if wav.ndim > 1:
                        wav = wav.mean(axis=1)

                    # normalize & quantize PCM_16 (tiết kiệm dung lượng)
                    if np.max(np.abs(wav)) > 0:
                        wav = wav/np.max(np.abs(wav))*0.95
                    wav_i16 = (wav * 32767.0).astype(np.int16)

                    # text
                    text = batch_dict.get("transcription", [None])[i] or batch_dict.get("text", [None])[i] or ""
                    text = text.strip()
                    if not (3 <= len(text) <= 500):
                        continue

                    # save wav
                    out_name = f"vi_{shard_idx:02d}_{written:06d}.wav"
                    out_path = os.path.join(WAV_DIR, out_name)
                    sf.write(out_path, wav_i16, target_sr, subtype="PCM_16")

                    # metadata row (XTTS: audio|text|speaker|language)
                    speaker = f"spk_{shard_idx:02d}"
                    metadata_lines.append(f"{out_name}|{text}|{speaker}|vi")

                    written += 1
                except Exception:
                    continue
            if written >= max_samples:
                break
    finally:
        if local_path and os.path.exists(local_path):
            try: os.remove(local_path)
            except: pass
        gc.collect()

    meta_path = os.path.join(DATASET_DIR, "metadata.csv")
    with open(meta_path, "w", encoding="utf-8") as f:
        f.write("\n".join(metadata_lines))

    print(f"✅ Shard {shard_idx+1}: wrote {written} samples → {meta_path}")
    return written

print("✅ Shard prep functions ready.")


In [ ]:
# ==== CELL 4: WRITE XTTS FINETUNE CONFIG (YAML) ====
import yaml, pathlib

OUT_DIR = "/kaggle/working/xtts_vietnamese"     # nơi ghi checkpoint/logs
CFG_PATH = f"{OUT_DIR}/config_ft.yaml"
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

config = {
    "model": "xtts_v2",                # model family
    "output_path": OUT_DIR,
    "run_name": "xtts_vi_stream",
    "logger": "tensorboard",

    # ==== DATASET ====
    "dataset_config": [
        {
            "formatter": "ljspeech",   # dùng formatter ljspeech: metadata + wavs
            "meta_file_train": f"{DATASET_DIR}/metadata.csv",
            "meta_file_val": f"{DATASET_DIR}/metadata.csv",   # tạm thời dùng chung
            "path": DATASET_DIR,       # base path
            "audio_dir": WAV_DIR,      # thư mục wavs
            "language": "vi",
            "n_val": 64,               # mỗi shard validation tạm 64 mẫu
        }
    ],

    # ==== AUDIO ====
    "audio": {
        "sample_rate": 22050,
        "num_mels": 80,          # XTTS dùng mel-80 nội bộ (không cần tự tính)
        "fft_size": 1024,
        "hop_length": 256
    },

    # ==== OPTIMIZATION ====
    "optimizer": {
        "type": "adamw",
        "lr": 5e-5,
        "betas": [0.9, 0.98],
        "eps": 1e-9,
        "weight_decay": 0.0
    },

    # ==== TRAINER ====
    "trainer": {
        "max_steps": 1500,       # sẽ bị ghi đè mỗi shard
        "eval_steps": 250,
        "save_step": 500,
        "log_step": 50,
        "precision": "fp16",
        "batch_size": 2,
        "grad_accum": 4,         # effective batch ~8
        "num_workers": 2,
        "pin_memory": True,
        "cudnn_benchmark": True,
        "seed": 42
    },

    # ==== LOSSES (để trainer tự cấu hình mặc định XTTS) ====
    "losses": {
        "use_xtts_defaults": True
    },

    # ==== FREEZE POLICY (nhẹ nhàng; có thể bỏ nếu muốn full) ====
    "freeze": {
        # gợi ý: cho phép GPT + decoder học; freeze phần ít quan trọng
        "freeze_encoder": False,
        "freeze_gpt": False,
        "freeze_decoder": False
    }
}

with open(CFG_PATH, "w") as f:
    yaml.safe_dump(config, f, sort_keys=False, allow_unicode=True)

print("✅ Wrote YAML:", CFG_PATH)
print(open(CFG_PATH).read())


In [ ]:
print("heellloo")
